# Tools & Libraries

In [4]:
# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
print(file_dir)
parent_dir = file_dir.parent
print(parent_dir)
sys.path.append(str(parent_dir))

C:\Users\stc\Documents\GitHub\PredictiveAnalyticsPython\stc
C:\Users\stc\Documents\GitHub\PredictiveAnalyticsPython


In [77]:
from pa_lib.job import request_job
from pa_lib.file import project_dir, load_bin
from pa_lib.data import desc_col
from pa_lib.vis import boxplot_histogram

In [6]:
import pandas as pd
import numpy as np

In [120]:
with project_dir("vkprog/predictions"):
    ek_list = load_bin("20200127_ek_list.feather")

11:57:13 [INFO] Started loading binary file ...
11:57:13 [INFO] Reading from file C:\Users\stc\data\vkprog\predictions\20200127_ek_list.feather
11:57:13 [INFO] ... finished loading binary file in 0.01s (0.02s CPU)


In [121]:
desc_col(ek_list)

,DTYPE,NULLS,UNIQUE
index,int64,0/12703,12703
Endkunde_NR,int64,0/12703,12703
Endkunde,object,0/12703,12130
HB_APG,object,286/12417,61
Agentur,object,7484/5219,1783
HB_Agentur,object,7527/5176,94
PLZ,int64,0/12703,2000
Ort,object,6/12697,1952
Net_2016,float64,121/12582,4506
Net_2017,float64,121/12582,4587


In [122]:
net_columns = [col for col in ek_list.columns if "Net" in col]
net_max = ek_list.loc[:, net_columns].fillna(0).max(axis="columns")
ek_list.loc[:, "Net_Max"] = net_max

In [123]:
ek_list.loc[:,net_columns+["Net_Max"]].sample(10)

,Net_2016,Net_2017,Net_2018,Net_2019,Net_2020,Net_Max
1568,1854.0,1902.0,525.0,950.0,0.0,1902.0
10685,8487.0,7399.0,0.0,0.0,0.0,8487.0
11286,0.0,0.0,580.0,0.0,0.0,580.0
8623,10200.0,0.0,59310.0,1401.0,0.0,59310.0
12320,0.0,556.0,54.0,0.0,0.0,556.0
2576,4650.0,0.0,0.0,0.0,0.0,4650.0
6134,1420.0,2750.0,4820.0,1140.0,0.0,4820.0
6597,0.0,0.0,1170.0,221771.0,0.0,221771.0
3276,5600.0,0.0,0.0,0.0,0.0,5600.0
10233,1753.0,4166.0,0.0,0.0,0.0,4166.0


In [124]:
ek_list.loc[:, "_HB_APG_"] = ek_list.HB_APG.fillna("NONE")

In [125]:
ranking_pro_vb = ek_list.groupby("_HB_APG_")["Net_Max"].rank(ascending=True,method='dense')

ek_list.loc[:,"ranking_01"] = ranking_pro_vb


In [126]:
desc_col(ek_list.loc[:,["ranking_01","Net_Max"]],det=True)

,DTYPE,NULLS,UNIQUE,MEM,RANGE
ranking_01,float64,0/12703,719,99.3 KB,"[1.0,719.0]"
Net_Max,float64,0/12703,8307,99.3 KB,"[0.0,16877409.0]"


In [127]:
ek_list.loc[:,["HB_APG"]+net_columns+["Net_Max","ranking_01"]].sample(10)

,HB_APG,Net_2016,Net_2017,Net_2018,Net_2019,Net_2020,Net_Max,ranking_01
3768,JLP,31657.0,30900.0,31346.0,34589.0,0.0,34589.0,207.0
8869,SAO,3684.0,0.0,3561.0,0.0,0.0,3684.0,115.0
5699,BON,17828.0,1400.0,2104.0,0.0,0.0,17828.0,488.0
983,RAM,491119.0,545221.0,1592153.0,747398.0,87809.0,1592153.0,54.0
105,CKT,0.0,3680.0,21695.0,2761.0,0.0,21695.0,307.0
10607,REM,0.0,2859.0,5074.0,0.0,0.0,5074.0,249.0
11567,DCI,0.0,0.0,4059.0,0.0,0.0,4059.0,217.0
4464,REM,8051.0,13562.0,19819.0,11881.0,0.0,19819.0,403.0
6744,DCI,0.0,522.0,308.0,387.0,0.0,522.0,46.0
9722,MOE,0.0,0.0,0.0,13096.0,0.0,13096.0,518.0


In [128]:
ek_list = pd.merge(
    ek_list,
    pd.DataFrame(ek_list.groupby(by="_HB_APG_")["ranking_01"].max())
    .reset_index()
    .rename(columns={"ranking_01": "max_rank"}),
    on="_HB_APG_",
    how="left",
).sort_values("ranking_01",ascending=False)

In [129]:
ek_list.loc[:,"f_score"] = ek_list.ranking_01/ek_list.max_rank

In [130]:
ek_list.head(10)

,index,Endkunde_NR,Endkunde,HB_APG,Agentur,HB_Agentur,PLZ,Ort,Net_2016,Net_2017,...,prob_KW,Insolvenz,last_CRM_Ktkt_date,VB_FILTER_VON,VB_FILTER_BIS,Net_Max,_HB_APG_,ranking_01,max_rank,f_score
346,346,121672,WWF Schweiz,MOE,Agentur Weber Guido,MBU,8010,Zürich,1421066.0,278967.0,...,0.534011,None,2019-12-05,NaT,NaT,1421066.0,MOE,719.0,719.0,1.000000
6907,6907,650094,John Kiss AG,MOE,None,None,8049,Zürich,0.0,0.0,...,0.207529,None,NaT,NaT,NaT,1220254.0,MOE,718.0,719.0,0.998609
353,353,564388,Swisscard AECS GmbH,MOE,NMPi,MOE,8810,Horgen,221397.0,708246.0,...,0.532810,None,2019-12-05,NaT,NaT,708246.0,MOE,717.0,719.0,0.997218
6252,6252,502896,Organizing Committee,MOE,None,None,6302,Zug,0.0,0.0,...,0.245264,None,2019-12-13,NaT,NaT,459863.0,MOE,716.0,719.0,0.995828
6609,6609,137571,Swiss Casinos Pfäffikon-Zürichsee,MOE,Swiss Casinos Services AG,MOE,8808,Pfäffikon SZ,32600.0,0.0,...,0.226110,None,2019-10-26,NaT,NaT,432376.0,MOE,715.0,719.0,0.994437
721,721,594924,USP Partner AG,MOE,None,None,8800,Thalwil,224641.0,411088.0,...,0.477851,None,2019-12-11,NaT,NaT,411088.0,MOE,714.0,719.0,0.993046
1012,1012,535254,Kieser Training Schweiz AG,MOE,ZipMedia GmbH,NYI,8005,Zürich,40433.0,2218.0,...,0.449460,None,2019-12-05,NaT,NaT,393658.0,MOE,713.0,719.0,0.991655
1525,1525,123750,Swiss Casinos Services AG,MOE,Propaganda Zürich AG,MOE,8001,Zürich,0.0,0.0,...,0.407989,None,2020-01-15,NaT,NaT,313867.0,MOE,712.0,719.0,0.990264
819,819,607732,MoneyPark AG,MOE,ACE 2 ACE outdoor media ag,NYI,8002,Zürich,36517.0,158679.0,...,0.467987,None,2019-10-27,NaT,NaT,309180.0,MOE,711.0,719.0,0.988873
118,118,606776,Twint AG,MOE,ZipMedia GmbH,PKL,8004,Zürich,300160.0,255884.0,...,0.592144,None,2019-12-05,NaT,NaT,300160.0,MOE,710.0,719.0,0.987483


In [ ]:
for vb in list(set(ek_list._HB_APG_)):
    print(88*"#")
    print(f"\n{vb}: {ek_list.loc[ek_list._HB_APG_ == vb, interesting].shape}")
    
    for interesting in ["Net_Max", "f_score"]:
        boxplot_histogram(ek_list.loc[ek_list._HB_APG_ == vb, interesting])

,DTYPE,NULLS,UNIQUE
HB_APG,object,286/12417,61
f_score,float64,286/12417,10246


In [135]:
for x in [
    ek_list.loc[
        :,
        ["HB_APG", "_HB_APG_"]
        + net_columns
        + ["prob_KW", "Net_Max", "ranking_01", "max_rank", "f_score"],
    ]
]:
    display(x.sample(10))
    display(desc_col(x))
    display(x.loc[x.HB_APG.isna(), :])

,HB_APG,_HB_APG_,Net_2016,Net_2017,Net_2018,Net_2019,Net_2020,prob_KW,Net_Max,ranking_01,max_rank,f_score
9978,REM,REM,0.0,0.0,0.0,690.0,0.0,0.106375,690.0,61.0,497.0,0.122736
161,CKT,CKT,0.0,3781.0,3731.0,5434.0,0.0,0.578803,5434.0,222.0,382.0,0.581152
8213,PAL,PAL,0.0,0.0,0.0,990.0,0.0,0.153733,990.0,131.0,690.0,0.189855
7444,CBA,CBA,0.0,0.0,457.0,0.0,0.0,0.178793,457.0,31.0,253.0,0.122530
4006,SFA,SFA,2296.0,0.0,0.0,0.0,0.0,0.316160,2296.0,154.0,411.0,0.374696
10528,PAL,PAL,8552.0,8664.0,15085.0,5100.0,0.0,0.095097,15085.0,566.0,690.0,0.820290
10780,JAN,JAN,0.0,0.0,1410.0,0.0,0.0,0.091652,1410.0,51.0,198.0,0.257576
7790,PAL,PAL,0.0,0.0,0.0,11715.0,0.0,0.164990,11715.0,545.0,690.0,0.789855
10851,BWU,BWU,12029.0,6723.0,6165.0,0.0,0.0,0.090902,12029.0,283.0,386.0,0.733161
8766,TRA,TRA,0.0,6707.0,5286.0,13500.0,3000.0,0.135490,13500.0,192.0,263.0,0.730038


,DTYPE,NULLS,UNIQUE
HB_APG,object,286/12417,61
_HB_APG_,object,0/12703,62
Net_2016,float64,121/12582,4506
Net_2017,float64,121/12582,4587
Net_2018,float64,121/12582,4553
Net_2019,float64,121/12582,4657
Net_2020,float64,121/12582,719
prob_KW,float64,0/12703,12608
Net_Max,float64,0/12703,8307
ranking_01,float64,0/12703,719


,HB_APG,_HB_APG_,Net_2016,Net_2017,Net_2018,Net_2019,Net_2020,prob_KW,Net_Max,ranking_01,max_rank,f_score
9901,None,NONE,2599468.0,882749.0,0.0,0.0,0.0,0.107814,2599468.0,264.0,264.0,1.000000
7886,None,NONE,194572.0,1098707.0,47320.0,0.0,0.0,0.162272,1098707.0,263.0,264.0,0.996212
6341,None,NONE,7470.0,0.0,0.0,1032750.0,0.0,0.240169,1032750.0,262.0,264.0,0.992424
10058,None,NONE,790052.0,26236.0,0.0,0.0,0.0,0.104960,790052.0,261.0,264.0,0.988636
4037,None,NONE,532938.0,0.0,0.0,0.0,0.0,0.315069,532938.0,260.0,264.0,0.984848
2682,None,NONE,0.0,0.0,47594.0,0.0,469310.0,0.349367,469310.0,259.0,264.0,0.981061
10300,None,NONE,450789.0,321905.0,0.0,0.0,0.0,0.099524,450789.0,258.0,264.0,0.977273
9668,None,NONE,0.0,0.0,419999.0,0.0,0.0,0.112608,419999.0,257.0,264.0,0.973485
11692,None,NONE,33150.0,193275.0,0.0,0.0,0.0,0.077416,193275.0,256.0,264.0,0.969697
10738,None,NONE,0.0,0.0,175587.0,0.0,0.0,0.092312,175587.0,255.0,264.0,0.965909
